In [2]:
import os
import json
import traceback
import streamlit as st
import pandas as pd
from src.mcqgenerator.utils import read_file,get_table_data
from src.mcqgenerator.MCQGenerator import genereate_evaluate_chain
from src.mcqgenerator.looger import logging
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
load_dotenv()

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
#Testing api key
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.getenv("GOOGLE-API-KEY"),
    temperature=0.3
)

response = llm.invoke("Say hello in Python")
print(response)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key = os.getenv("GOOGLE-API-KEY")
)

In [ ]:
from src.mcqgenerator.utils import read_file,get_table_data
from src.mcqgenerator.looger import logging

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [ ]:
quiz_generation_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content = "You are an expert MCQ quiz maker. You have to create the quiz based on user input and given instructions."
        ),
        HumanMessage(
            content = """
                Please create {number} muliple choice questions in a {tone} tone based on the following text.
                **Text:**
                {text}
                
                **Instructions:**
                1. Ensure that all questions are from given text only.
                2. Ensure that the questions are not repeated.
                3. Format your entire response as a JSON object that strictly follows below given scheme.
                {response_json}
                4. Provide only the question and answers for the questions in the response.
                5. Do not repeat given text in your response.
            """
        )
    ]    
)

In [ ]:
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [ ]:
print(quiz_generation_prompt.input_variables)

In [ ]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
    "2":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
    "3":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
}

In [ ]:
review_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content ="You are an expert English grammarian and writer. You have to evaluate the given quiz as per instructions given below"
        ),
        HumanMessage(
            content="""Please review the multiple choice quiz intendede for {subject} sy=tudents.
                **Quiz for review:**
                '''json
                {quiz}
                '''

                **Your Task:**
                1. Evaluate the complexity of the questions.
                2. Assess if the quiz is appropriate for students cognitive and quantitatib=ve ability.
                3. If the quiz is not suitable, then rewrite the questions and adjust the tone to perfectly fit students abilities.
                4. Provide a brief analysis of your changes or your approval of the original quiz.
                
            """
        )
    ]
)


In [ ]:
review_chain = review_prompt | llm | StrOutputParser()

In [ ]:
genereate_evaluate_chain =(
    RunnablePassthrough.assign(quiz = quiz_chain) | 
    RunnablePassthrough.assign(review = review_chain)
)

In [ ]:
file_path = "P:\Code\Gen_AI_course_ineuron\MCQ_GEN_AI\data.txt"
with open(file_path,"r") as file:
    TEXT = file.read()

In [ ]:
print(TEXT)

In [ ]:
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "simple"

In [ ]:
with get_openai_callback() as cb:
                response = quiz_chain.invoke(
                    {
                        "text": TEXT,
                        "number":NUMBER,
                        "subject": SUBJECT,
                        "tone":TONE,
                        "response_json": json.dumps(RESPONSE_JSON)
                    }
                )

In [ ]:
print(response)

In [ ]:
import json

# Clean the string by removing the markdown formatting
cleaned_string = response.strip("```json\n").strip("```")

# Now, parse the cleaned string into a Python dictionary
quiz_dict = json.loads(cleaned_string)

print("Successfully parsed the quiz!")

In [ ]:
print(f"Total Tokens Used: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost:.4f}")

In [ ]:
import pandas as pd

# The actual questions are inside the 'quiz' key
table_data = quiz_dict['quiz']

# This will now work correctly
df = pd.DataFrame(table_data)

print(df)

In [ ]:
print(df)

In [ ]:
quiz_string = response
cleaned_string = quiz_string.strip("```json\n").strip("`")
final_mcqs = json.loads(cleaned_string)
final_mcqs
print(json.dumps(final_mcqs, indent=4))

In [ ]:
quiz_table_data = []
for key,value in final_mcqs.items():
    mcq = value["question"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    answer = value["answer"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Answer":answer})


In [ ]:
quiz_table_data

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)